# Исследование надежности заемщиков


**Цель исследования**

1. Провести предобработку данных, сформировать новые категории на основе имеющихся данных

2. Ответить на вопросы (на основе сводной таблицы):

- Есть ли зависимость между количеством детей и возвратом кредита в срок?
- Есть ли зависимость между семейным положением и возвратом кредита в срок?
- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
- Как разные цели кредита влияют на его возврат в срок?

## Обзор данных

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')


In [3]:
try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('/content/data.csv')

Выведем первые 20 строк датафрейма.

In [4]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Выведем основную информацию о датафрейме.

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

In [6]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

В двух столбцах есть пропущенные значения. Один из них — `days_employed`. Пропуски в этом столбце мы обработаем на следующем этапе. Другой столбец с пропущенными значениями — `total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце можно медианным значением по каждому типу из столбца `income_type`. 

In [7]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

В наших данных есть аномальные значения — отрицательное количество дней трудового стажа в столбце `days_employed`. Заменим все отрицательные значения положительными с помощью метода `abs()`.

In [8]:
data['days_employed'] = data['days_employed'].abs()

Посмотрим на столбец `children`.

In [9]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

В столбце `children` есть два аномальных значения. Удалим строки, в которых встречаются такие аномальные значения, из нашей таблицы.

In [10]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

In [11]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

Заполним пропуски в столбце `days_employed` медианными значениями по каждого типа занятости `income_type`.

In [12]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

In [13]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

Заменим вещественный тип данных в столбце `total_income` на целочисленный с помощью метода `astype()`.

In [14]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

Обработаем неявные дубликаты в столбце `education`. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведем их к нижнему регистру. 

In [15]:
data['education'] = data['education'].str.lower()

Посмотрим на явные дубликаты и удалим их при наличии.

In [16]:
data.duplicated().sum()

71

In [17]:
data = data.drop_duplicates()

### Категоризация данных

На основании диапазонов, указанных ниже, создадим в нашей таблице столбец `total_income_category` с категориями:

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.


In [18]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [19]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

Посмотрим на уникальные значения в столбце `purpose`.

In [20]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

На основании данных из столбца `purpose` сформируем новый столбец `purpose_category`, в который войдут следующие категории:

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.



In [21]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [22]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

## Исследование данных и ответы на вопросы

### Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [24]:
data['debt'].unique()


array([0, 1])


Значения в столбце `debt` — `0` (кредит возвращен в срок) и `1` (есть долг по кредиту). 



In [25]:
# группируем данные по значениям в столбце children и считаем общую сумму невозвратов
# в каждой группе
debts_grouped_by_children = data.groupby('children')['debt'].sum()
debts_grouped_by_children.to_frame()


,debt
children,
0,1063
1,444
2,194
3,27
4,4
5,0


После группировки данных по столбцу, где содержится информация о количестве детей, видно, что чем меньше у клиента детей, тем большее количество кредитов возвращено не в срок. Так, при отсутствии детей количество долгов по кредитам равно 1063, по мере увеличения числа детей этот показатель падает, при наличии пяти детей равняется нулю.

**Итак, наша гипотеза состоит в том, что между наличием у клиента детей и возвратом кредита в срок существует зависимость: чем больше у клиента детей, тем выше вероятность, что кредит будет возвращен в срок.**

Данная гипотеза нуждается в проверке, т. к. необходимо установить, какова доля невозвратов в каждой группе, поскольку учет только абсолютного числа возвратов без учета доли в каждой группе заемщиков может быть не показательным (например, если заемщиков без детей существенно больше, чем с детьми).


In [29]:
# строим сводную таблицу 
children_pivot_table = pd.pivot_table(data,
                                      index = 'children',
                                      values = ['debt'], 
                                      aggfunc = {sum, len})

# добавляем в таблицу столбец с процентом просрочек по группам
children_pivot_table['debt_share'] = (children_pivot_table['debt']['sum'] 
                                      / children_pivot_table['debt']['len']) * 100

# переименовываем столбцы
children_pivot_table.set_axis(
    ['Количество клиентов', 'Количество просрочек', 'Процент просрочек'], 
    axis=1, inplace=True)

children_pivot_table



,Количество клиентов,Количество просрочек,Процент просрочек
children,,,
0,14091,1063,7.543822
1,4808,444,9.234609
2,2052,194,9.454191
3,330,27,8.181818
4,41,4,9.756098
5,9,0,0.000000


Как следует из сгруппированных данных, доля просрочек наиболее высокая среди семей с четырьмя, двумя и одним ребенком. Доля клиентов с просрочками по кредитам в группе без детей меньше на два процентных пункта. Группа клиентов с пятью детьми является малочисленной, и следовательно, нерепрезентативной.


**Вывод: Гипотеза о том, что чем больше у клиента детей, тем выше вероятность, что кредит будет возвращен в срок, не подтвердилась. Судя по сгруппированным данным, у клиентов без детей вероятность возврата кредита в срок оказывается выше, а доля просроченных кредитов — ниже.


### Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [28]:
# группируем данные по столбцу family_status
# и выводим количество должников по группам
debts_grouped_by_family_status = data.groupby('family_status')['debt'].sum()
debts_grouped_by_family_status.to_frame()

,debt
family_status,
Не женат / не замужем,273
в разводе,84
вдовец / вдова,63
гражданский брак,385
женат / замужем,927


Судя по сгруппированным данным, число должников выше среди клиентов, которые состоят в официальном браке. Посчитаем долю невозвратов в каждой группе.

In [30]:
# строим сводную таблицу 
family_status_pivot_table = pd.pivot_table(data,
                                           index = 'family_status',
                                           values = ['debt'], 
                                           aggfunc = {len, sum})

# добавляем в таблицу столбец с процентом просрочек по группам
family_status_pivot_table ['debt_share'] = (
    family_status_pivot_table['debt']['sum']
    / family_status_pivot_table ['debt']['len']) * 100

#переименовываем столбцы
family_status_pivot_table.set_axis(
    ['Количество клиентов', 'Количество просрочек', 'Процент просрочек'], 
    axis=1, inplace=True)

family_status_pivot_table


,Количество клиентов,Количество просрочек,Процент просрочек
family_status,,,
Не женат / не замужем,2796,273,9.763948
в разводе,1189,84,7.064760
вдовец / вдова,951,63,6.624606
гражданский брак,4134,385,9.313014
женат / замужем,12261,927,7.560558


Судя по сгруппированным данным, наиболее высокий процент просрочек — среди клиентов, которые не состоят в браке или состоят в гражданском (неофициальном) браке. Наименьший процент — среди вдовцов (вдов). Возможно, это объясняется меньшим количеством клиентов-вдовцов (вдов) по сравнению с другими группами. Средний показатель — среди клиентов, состоящих в законном браке и разведенных.


Вывод: Наличие партнера и семейный статус заемщика влияют на вероятность возврата кредита в срок. Наибольшая вероятность — среди клиентов, которые разведены или состоят в законном браке, в этих группах меньше всего процент просрочек по кредитам. Среди клиентов, которые не состоят в браке или состоят в гражданском браке, вероятность возврата кредита в срок ниже, доля просрочек в этих группах выше.

### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [31]:
# группируем данные по столбцу total_income_category 
# и считаем количество должников в каждой группе
debts_grouped_by_total_income_category = data.groupby('total_income_category')['debt'].sum()
debts_grouped_by_total_income_category.to_frame()


,debt
total_income_category,
A,2
B,354
C,1353
D,21
E,2


Больше всего должников — в категории С. Посмотрим на общее количество клиентов в каждой категории.

In [33]:
data.total_income_category.value_counts().to_frame()

,total_income_category
C,15921
B,5014
D,349
A,25
E,22


Наибольшее количество клиентов — в категориях С (доход от 50 тыс. до 200 тыс. рублей) и B (доход от 200 тыс. до 1 млн рублей). Построим сводную таблицу.

In [34]:
# строим сводную таблицу 
income_pivot_table = pd.pivot_table(data,
                                    index = 'total_income_category',
                                    values = ['debt'], 
                                    aggfunc = {len, sum})

#считаем долю должников в процентах и добавляем столбец
income_pivot_table ['share'] = (income_pivot_table['debt']['sum'] 
                                / income_pivot_table['debt']['len']) * 100

#сортируем значения по убыванию по столбцам debt, len
income_pivot_table = income_pivot_table.sort_values(by=('debt', 'len'), 
                                                    ascending = False)

# #переименовываем столбцы
income_pivot_table.set_axis(
    ['Количество клиентов', 'Количество просрочек', 'Процент просрочек'], 
    axis=1, inplace=True)

income_pivot_table



,Количество клиентов,Количество просрочек,Процент просрочек
total_income_category,,,
C,15921,1353,8.498210
B,5014,354,7.060231
D,349,21,6.017192
A,25,2,8.000000
E,22,2,9.090909


Для анализа будем опираться на данные по двум первым (по численности) категориям клиентов как наиболее репрезентативным. Как следует из таблицы, наиболее высокий процент просрочек — среди клиентов категории C (доход от 50 тыс. до 200 тыс. рублей). У группы клиентов с более высоким уровнем дохода B (доход от 200 тыс. до 1 млн рублей) доля просрочек ниже.

Вывод: Уровень дохода оказывает влияние на вероятность возврата кредита в срок. Среди клиентов с более высоким уровнем дохода ниже процент просрочек по кредитам.

### Как разные цели кредита влияют на его возврат в срок?

In [36]:
debts_grouped_by_purpose_category = data.groupby('purpose_category')['debt'].sum()
debts_grouped_by_purpose_category.to_frame()



,debt
purpose_category,
операции с автомобилем,400
операции с недвижимостью,780
получение образования,369
проведение свадьбы,183


Мы сгруппировали данные по категориям целей кредита и посчитали количество должников в каждой группе. По абсолютному покзателю лидируют операции с недвижимостью. Рассчитаем долю должников в каждой группе и построим сводную таблицу.

In [37]:
# строим сводную таблицу
purpose_pivot_table = pd.pivot_table(data,
                                     index = 'purpose_category',
                                     values = ['debt'], 
                                     aggfunc = {len, sum})

# добавляем в таблицу столбец с процентом просрочек по группам
purpose_pivot_table ['share'] = (purpose_pivot_table ['debt']['sum'] 
                                 / purpose_pivot_table ['debt']['len']) * 100

#переименовываем столбцы
purpose_pivot_table.set_axis(
    ['Количество клиентов', 'Количество просрочек', 'Процент просрочек'], 
    axis=1, inplace=True)

purpose_pivot_table



,Количество клиентов,Количество просрочек,Процент просрочек
purpose_category,,,
операции с автомобилем,4279,400,9.347978
операции с недвижимостью,10751,780,7.255139
получение образования,3988,369,9.252758
проведение свадьбы,2313,183,7.911803


В процентном отношении операции с недвижимостью оказались на последнем месте по доле должников. Наибольший процент просрочек — в группе "операции с автомобилем".


Вывод: Между целью кредита и вероятностью его возврата в срок существует зависимость. Вероятность возврата кредита в срок выше среди клиентов, у которых цель кредита — приобретение недвижимости или иные операции с недвижимостью. В группе клиентов, которые берут кредит на покупку автомобиля, процент просрочек выше, а вероятность возврата кредита в срок — ниже.

## Вывод про проекту

<span style='background :orange'> В ходе анализа было установлено, что на вероятность возврата кредита в срок могут влиять:
1. Наличие и количество детей у заемщика: чем больше у клиента детей, тем больше вероятность просрочки по кредиту.
2. Семейный статус заемщика: вероятность просрочки выше, если клиент не состоит в браке или состоит в гражданском браке.
3. Уровень дохода заемщика: чем выше уровень дохода, тем ниже вероятность просрочки по кредиту.
4. Цель получения кредита: если целью заемщика являются операции с недвижимостью (покупка квартиры, приобретение коммерческой недвижимости и др.), то вероятность просрочки ниже; если заемщик берет кредит на покупку автомобиля или получение образования, вероятность просрочки выше. 
    </span>

